# Titanic: Machine Learning from Disaster
## CRISP-DM: Data Preparation
**Autor:** Wanderson Marques - wdsmarques@gmail.com

Esse Jupyter Notebook contém o **pré-processamento** do conjunto de dados Titanic. Considerando a metodologia CRISP-DM, essa atividade refere-se à terceira fase, a preparação dos dados. 

<img src="imgs/dataPreparation.jpg" />

### Carregar bibliotecas

In [1]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

### Carregar dataset

In [2]:
dataset = pd.read_csv('datasets/train.csv')
dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Descartar os atributos textuais e identificador dos passageiros
Posteriormente um trabalho de mineração textos poderia ser realizado a fim de extrair alguma informação potencialmente relevante.

In [3]:
dataset.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1, inplace=True)

### Separar conjuntos de treino (70%) e validação (30%)

O conjunto de validação será utilizado posteriormente, para verificar a capacidade preditiva do modelo. Ele é isolado para que não receba nenhum tipo de viés.

In [4]:
train, valid = train_test_split(dataset.copy(), test_size=0.3, random_state=1)

### Gravar conjuntos de validação

In [5]:
valid.to_csv('datasets/valid.csv', index=False)

### Tratar valores nulos

São tratamentos comuns para valores nulos:
- Exclusão do atributo (caso ele seja nulo para grande parte das instâncias)
- Exclusão da instância (caso ela seja nula para grande parte dos atributos)
- Imputação por estatísticas simples, como média, mediana ou moda (podem ser calculadas para sub-amostras)
- Imputação por regressão e modelos preditivos

In [6]:
train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         127
SibSp         0
Parch         0
Fare          0
Embarked      1
dtype: int64

In [7]:
# Para as instâncias onde Embarked é nulo, imputar a moda (S)
train.loc[train['Embarked'].isnull(), 'Embarked'] = (train.mode())['Embarked'][0]

In [8]:
# Para as instâncias onde Age é nulo, imputar a média (29.91)
train.loc[train['Age'].isnull(), 'Age'] = train.mean()['Age']

In [9]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

### Tratar outliers

São tratamentos comuns para valores nulos:

- Exclusão da instância
- Correção por estatísticas simples ou modelos preditivos

In [10]:
# Nesse caso serão tratados apenas os outliers onde Fare > 300
train.loc[train['Fare'] > 300]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
679,1,1,male,36.0,0,1,512.3292,C


In [11]:
mediaPrimeiraClasse = train.loc[train['Pclass'] == 1].mean()['Fare']
train.loc[train['Fare'] > 300, 'Fare'] = mediaPrimeiraClasse

In [12]:
train.loc[train['Fare'] > 300]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked


### Converter variáveis categóricas em dummies

Cada valor possível para uma variável categórica devem ser transformado em um atributo binário (0 ou 1)

In [13]:
train = pd.get_dummies(train)

In [14]:
# Evitar dummy trap excluindo valores de referência
train.drop(['Sex_male', 'Embarked_S'], axis=1, inplace=True)

### Balancear classes

Em problemas de classificação onde as classes (0 e 1 para Survived) estão desbalanceadas é comum que seja feito o balanceamento para evitar um modelo enviesado. Esse balanceamento pode ser feito limitando a quantidade de instâncias pela classe de menor representatividade ou utilizando técnicas de sobre-amostragem (oversampling). 

In [15]:
# Classes desbalanceadas
train['Survived'].value_counts()

0    396
1    227
Name: Survived, dtype: int64

In [16]:
# Separar dados em variáveis X (preditoras) e y (predita)
X = train.drop(['Survived'], axis=1)
y = train['Survived']

In [17]:
# Realizar oversampling
sm = SMOTE()
X_res, y_res = sm.fit_sample(X, y)

y = pd.DataFrame(y_res, columns=['Survived'])

In [18]:
# Classes balanceadas
y['Survived'].value_counts()

1    396
0    396
Name: Survived, dtype: int64

### Padronizar atributos

É comum que cada atributo possua uma escala diferente, porém isso pode ser um problema para a maioria dos modelos preditivos. Portanto, é necessário transformar os dados, colocando todos os atributos em uma mesma escala.

In [19]:
# Preparar o scaler
# Outros scalers: RobustScaler, MinMaxScaler
scaler = StandardScaler()
scaler.fit(X)

# Salvar o scaler para uso futuro
joblib.dump(scaler, filename='models/scaler.pkl')

['models/scaler.pkl']

In [20]:
# Transformar os dados
X = pd.DataFrame(scaler.transform(X_res), columns=train.drop(['Survived'], axis=1).columns)

train = pd.concat([X, y], axis=1)

### Gravar conjuntos de treino pré-processado

Após a finalização do pré-processamento, os dados transformados são salvos.

In [21]:
train.head()

,Pclass,Age,SibSp,Parch,Fare,Sex_female,Embarked_C,Embarked_Q,Survived
0,0.802787,-9.864448e-01,-0.471492,-0.47646,-0.376645,1.367833,2.136001,-0.301775,0
1,-0.405272,-1.570346e-01,0.491588,-0.47646,-0.151489,1.367833,2.136001,-0.301775,1
2,0.802787,2.678779e-16,-0.471492,-0.47646,-0.531501,-0.731083,-0.468165,-0.301775,0
3,0.802787,-7.602420e-01,-0.471492,-0.47646,-0.485487,-0.731083,-0.468165,-0.301775,0
4,-0.405272,2.678779e-16,-0.471492,-0.47646,-0.717819,-0.731083,-0.468165,-0.301775,0


In [22]:
train.to_csv('datasets/train-preprocessado.csv', index=False)